In [ ]:
%matplotlib widget

from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

Finally we arrived to this stage. Previously, we have used the one vs one and one vs rest strategies to convert our multi classification problem to multiple binary classification problems. We also tried nearest neighbor techniques like knn classifier and weighted knn classifier. We have shown that these previous strategies are usually good when used in the proper circumstances. We also showcased many flaws with previous methods. These range from accuracy issues, numeric instability, and performance problems. Today, we will showcase a proper softmax classification method that will solve all of these previous issues if used correctly. We also have shown the softmax derivation to get some intuition in how the loss function will work. Leaving the details till later, we start using some more complicated datasets like handwritten digits. This is known as the Optical recognition of handwritten digits dataset provided by Scikit. With this, we can check how well a basic softmax loss will fare against this dataset. For now, let's start by visualizing some samples of the dataset.

In [ ]:
# load the full dataset and display it
dataset = load_digits() # make sure to shuffle the data so no local patterns emerge
feature_names = dataset.feature_names
target_names = dataset.target_names
data = dataset.data
target = dataset.target

In [ ]:
print("Feature names:", feature_names)
print("Target Categories:", target_names)
print("Features: ", data.shape, data.dtype)
print("Classes:", target.shape, target.dtype)

print("Features: ", data)
print("Classes:", target)
print("Feature range:", np.min(data).astype(np.int32), np.max(data).astype(np.int32))
print("Feature type:",data.dtype)
print("Target type:",target.dtype)

In [ ]:
fig, axes = plt.subplots(8,8,figsize=(6,6))

for i in range(8):
    for j in range(8):
        axes[i][j].imshow(data[i*8+j].reshape((8,8)), interpolation='none', cmap=cm.Greys)
        axes[i][j].axis('off')
plt.suptitle("Handwritten digits 8x8 Greyscale (64 images)")
plt.show()

Here are the first 64 images in the dataset. The images are only 8x8 so they're very low quality but it's still very clear to us what each digit the image represents. Let's split the dataset now.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=1) # 33 % split
print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

In [ ]:
def feature_scale(data_unscaled, scaled=True):
    if scaled == False:
        return data_unscaled

    max_value = np.max(data_unscaled)

    data_scaled = np.array(data_unscaled)
    data_scaled = data_scaled / max_value
    # min max normalization
    
    return data_scaled

data = feature_scale(data, scaled=True)
print(data)

As usual, we want to feature scale the data in order for gradient descent to perform well. Normally, we would use std normalization as it considers each feature separately and scales the features well. However, here we are dealing with 0-16 values as intensities for the digits. Each feature is a pixel in the image. From this, it's clear that there are many values to represent a specific digit on the image. It could be slanted, rotated, streched, etc. The image however will be completely different even though the digit will be the exact same. So it's ideal to understand that each specific feature is actually useless in the full context of all features. It's the way the different pixels are connected to each other is what matters. With this, the best thing to do is to feature scale the entire image at one by dividing by the full range of the data. This will scale the data between 0 and 1 which will work fine with gradient descent. This is similar to min max normalization.

In [ ]:
def add_bias_weight(data):
    m = data.shape[0]
    ones_feature = np.ones(m).reshape((-1,1)) # create a single feature of ones
    data_bias = np.hstack([data, ones_feature])
    return data_bias

data = add_bias_weight(data)
print(data)

From here on out, we need to drastically simplify our equations. Since softmax is an expensive function and we are dealing with multiple weights per category, it helps to simplify the model as much as possible. It turns out that to simplify we can actually remove the bias term from the equations and put it in the weights instead. The feature associated with the bias weight all become 1. This is a small trick to still keep the bias in the model, and it simplifies the model quite a bit. We can now remove the bias term completely. 

In [ ]:
def softmax(x):
    ez = np.exp(x)
    return ez / np.sum(ez, axis=1).reshape((-1,1))

Prediction will now go through the softmax function. Remember that the prediction now gives a vector of probabilities for each example. If we consider over all examples, we now get a matrix $m \times k$ where $m$ is the training examples and $k$ is the categories. The softmax is defined as $a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} $. We just perform this over each example and each category.

In [ ]:
def predict(w, x, epsilon=1e-15):
    prediction = np.matmul(x, w) # m k
    prediction = np.clip(prediction, epsilon, 1 - epsilon) # clip to avoid overflow issues
    return softmax(prediction) # pass it through a softmax

We also clip the prediction to avoid overflow issues. This is also something we did before passing the value in the sigmoid. Same pattern repeats here. The cost function here is very similar to logistic loss. In logistic loss, we used $-log(p1)$ to compute the loss for probability $p1$. Since there are multiple probabilities, we need weights for each probability value. The loss becomes 
\begin{equation}
  L(\mathbf{p},y)=\begin{cases}
    -log(p_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(p_k), & \text{if $y=k$}
  \end{cases}
\end{equation}
Then the full cost function becomes
$$
J(\mathbf{w},b) = -\frac{1}{m} \sum_{i=1}^{m} \sum_{j=1}^{k}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{p=1}^k e^{z^{(i)}_p} }
$$
The $1\left\{y^{(i)} == j\right\}$ is an indicator function where if $y^{(i)} == j$ then the expression becomes a $1$ or $0$ otherwise. Of course, we want to fully vectorize the cost function and the gradients for softmax. This is hard with the indicator function. Instead of using the indicator function we use a one hot encoded matrix $(O_{ij})$ to achieve the same output. We one hot the target labels to get the one hot matrix. Then, the new cost function simplifies to $$J(\mathbf{w},b) = -\frac{1}{m}  \sum_{i=1}^{m} \sum_{j=1}^{k}  O_{ij} \log \frac{e^{z^{(i)}_j}}{\sum_{p=1}^k e^{z^{(i)}_p} }$$

In [ ]:
def onehot_matrix(a):
    categories = np.unique(a).shape[0]
    return np.eye(categories)[a]

Here, we see that if we use one hot matrix, the code simplifies a lot just like the equation. 

In [ ]:
def log_loss_cost(w,x,y):
    onehot = onehot_matrix(y) # m k
    softmax_prob = predict(w, x) # m k
    overk = onehot * np.log(softmax_prob) # m k
    overm = np.sum(overk, axis=1) # m
    return -np.mean(overm) # 1

Now, computing the gradient is tedious as there are many derivatives to take. The final result once we compute the gradient however is very simple. The gradient with respect to a specific weight is $\frac{\partial L}{\partial w_{ji}} = x_j( \^{y_i} - y_i)$. $i$ is the $i$-th category while $x_j$ refers to the $j$-th feature of the $x$ input vector. $\^{y_i}$ is the $i$-th predicted probability while $y_i$ is the observed label. This is the gradient of the categorical loss function and it simplifies a lot once you compute it. Of course, this is for a single weight, so we need to apply this over all the weights per each category and over all the training examples to get the average gradient. 

In [ ]:
def compute_gradients(w,x,y):
    pass

In [ ]:
def show_cost_graph(costs, title, color):
    iterations = costs.shape[0]
    iteration_array = np.arange(0, iterations, dtype=np.int32)
    
    # graph the cost after updating the model
    fig, cost_graph = plt.subplots(layout='constrained')
    
    cost_graph.set_xlabel("Current Iteration")
    cost_graph.set_ylabel("Cost")
    
    cost_graph.set_title(title)
    
    cost_graph.plot(iteration_array, costs, color=color)

In [ ]:
def gradient_descent(x, y, gradient_func, cost_func, learning_rate=0.01, max_iterations=1000):
    y = y.reshape((-1,1))
    m = x.shape[0] # number of training examples
    n = x.shape[1] # number of features
    k = np.unique(y).shape[0] # number of categories

    w = np.zeros((n,k)) # n by k weights
    # initialize model parameters to zeroes

    costs = np.empty(0)

    for i in range(max_iterations):
        dw = gradient_func(w,x,y)
        
        w -= learning_rate * dw
        # update the weights
        
        current_cost = cost_func(w,x,y)
        costs = np.append(costs, current_cost)
        # add to array for visualization
    return w, costs
weights, costs = gradient_descent(data, target, compute_gradients, log_loss_cost, learning_rate=0.1, max_iterations=10000)
show_cost_graph(costs, "Logistic Regression with Softmax on Handwritten Digits", "blue")